In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style='color:red'>ch06_RNN기반의 Seq2Seq(스마트번역기)</span>

 - Google Neutral machine Translation (GNMT)
 - RNN 기반의 Sequence to Sequence 방식
 - 인코더(입력)/디코더(모법출력)연결구조
 * 응용분야: 자연어

# 패키지 및 하이퍼 파라미터

In [3]:
import pandas as pd
import numpy as np
from time import time
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

#하이파라미터
MY_HIDDEN = 128
MY_EPOCH  = 1000

# 학습데이터

In [8]:
raw = pd.read_csv('data/translate.csv',header=None)
eng_kor = raw.values.tolist() #데이터프레임을 list로 변환
print(eng_kor[:3])
print('영-한 번역데이터 갯수: ', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영-한 번역데이터 갯수:  110


# 영어 알파벳과 한글 문자 리스트 만들기

In [20]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
k_alpha = pd.read_csv('data/korean.csv',header=None)[0].tolist()
k_alpha == k_ch #순서와 내용이 같은지 여부

True

In [22]:
from collections import Counter
list1 = ['가','나','다']
list2 = ['나','다','가']
print(list1 == list2)
print(Counter(list1)==Counter(list2))

False
True


In [24]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha) # 171(원핫인코딩 사이즈)
print('전체 알파벳 갯수(원핫인코딩 사이즈) : ', alpha_total_size)

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩 사이즈) :  171


# 문자당 num을 갖는 dict

In [26]:
char_to_num = {}
for i, c in enumerate(alpha):
#     print(i,c)
    char_to_num[c] = i
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [27]:
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [28]:
# 숫자->문자/ 문자->숫자
print('숫자 -> 문자 : ', alpha[5])
print('문자 -> 숫자 : ', char_to_num['c'])


숫자 -> 문자 :  c
문자 -> 숫자 :  5


In [32]:
data = eng_kor[0]
print (data)
#char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d']
print('인코더 입력 : ', [char_to_num[ch] for ch in data[0]])
print('디코더 입력 : ', [char_to_num[ch] for ch in 'S'+data[1]])
print('디코더 출력 : ', [char_to_num[ch] for ch in data[1]+'E'])

['cold', '감기']
인코더 입력 :  [5, 17, 14, 6]
디코더 입력 :  [0, 32, 46]
디코더 출력 :  [32, 46, 1]


In [34]:
#원핫인코딩방법 1: pd.get_dummies([5,17,14,6]) - 이 코드에서 적용 불가 
pd.get_dummies([5,17,14,6])

,5,6,14,17
0,1,0,0,0
1,0,0,0,1
2,0,0,1,0
3,0,1,0,0


In [37]:
#원핫인코딩 방법2 : to_categorical([5,3,7],num_classes=10)
to_categorical([5,3,7], num_classes=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [39]:
#원핫인코딩 방법3 : np.eye(10)
np.eye(10)[[5,3,7]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

# 인코더 입력, 디코더 입력, 디코더 출력
* 인코더 입력(원핫 인코딩 ㅇ), 디코더입력(원핫인코딩ㅇ), 디코더 출력(원핫인코딩x)

In [79]:
def encoding(eng_kor=eng_kor):
    '인코더입력, 디코더입력, 디코더출력 데이터를 return'
    enc_in = [] # 인코더 입력
    dec_in = [] # 디코더 입력
    dec_out = [] # 디코더 출력
    for data in eng_kor:
        # 인코더 입력 데이터 (영어 -> 숫자 -> 원핫인코딩)
        eng = [char_to_num[ch] for ch in data[0]]
        #print(data[0], eng)
        #eng_one = to_categorical(eng, num_classes=alpha_total_size)#alpha_total_size:171
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)
        # 디코더 입력 데이터 ('S한글' -> 숫자 -> 원핫인코딩)
        kor = [char_to_num[ch] for ch in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        # 디코더 출력 ('한글E' -> 숫자 )
        kor = [char_to_num[ch] for ch in data[1]+'E']
        dec_out.append(kor)
    # 인공신경에 넣을 데이터이므로 numpy 배열로 전환
    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    dec_out = np.array(dec_out)
    #print(enc_in.shape, dec_in.shape, dec_out.shape)
    return enc_in, dec_in, dec_out

sample = [['wood', '나무'],
          ['word','단어']]

In [80]:
#RNN 분류분석 : 타겟변수를 원핫인코딩(시스템에게 원핫인코딩 의뢰)
X_enc, X_dec, y_dec = encoding(sample)
X_enc.shape, X_dec.shape, y_dec.shape

((2, 4, 171), (2, 3, 171), (2, 3))

In [81]:
y_dec

array([[ 48,  83,   1],
       [ 61, 114,   1]])

# 축증가

In [82]:
y_dec.reshape(2,3,1) #방법1

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [83]:
np.expand_dims(y_dec, axis=-1)#방법2

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [84]:
y_dec[...,np.newaxis]#방법3

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [85]:
y_dec[:,:,None]#방법4

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

# 전체 번역데이터(독립변수, 타겟변수)

In [87]:
X_enc, X_dec, y_dec = encoding(eng_kor)
Y_dec = np.expand_dims(y_dec, axis = -1)
#Y_dec = y_dec[...,np.newaxis]
#Y_dec = y_dec.reshape(-1,3,1)
X_enc.shape, X_dec.shape, y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3))

# 모델구현

In [97]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 윗출력, state_c, state_h
_, state_h, state_c = LSTM(
                    units=MY_HIDDEN,
                    # return_sequences=False 윗출력 안받음
                    return_state=True, 
            )(ENC_IN)
# 인코더와 디코더를 연결할 link
link = [state_h, state_c]
# 디코더 구현 : return_sequences=True 위로 올라는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              # return_state=False
              return_sequences=True)(DEC_IN,
                                    initial_state=link)
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

#모델
model = Model(inputs = [ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 4, 171)]     0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_13 (LSTM)                 [(None, 128),        153600      ['input_14[0][0]']               
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

# 학습과정 설정 및 학습하기

In [98]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])
start = time()
hist = model.fit([X_enc, X_dec], Y_dec,
                epochs = MY_EPOCH,
                verbose=2)
end = time()
print('학습시간 : ',{end-start})

Epoch 1/500
4/4 - 1s - loss: 5.1182 - accuracy: 0.2121 - 1s/epoch - 310ms/step
Epoch 2/500
4/4 - 0s - loss: 4.9606 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 3/500
4/4 - 0s - loss: 4.1636 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 4/500
4/4 - 0s - loss: 3.4663 - accuracy: 0.3333 - 17ms/epoch - 4ms/step
Epoch 5/500
4/4 - 0s - loss: 3.4130 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 6/500
4/4 - 0s - loss: 3.3752 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 7/500
4/4 - 0s - loss: 3.3438 - accuracy: 0.3333 - 19ms/epoch - 5ms/step
Epoch 8/500
4/4 - 0s - loss: 3.3170 - accuracy: 0.3333 - 19ms/epoch - 5ms/step
Epoch 9/500
4/4 - 0s - loss: 3.2902 - accuracy: 0.3333 - 17ms/epoch - 4ms/step
Epoch 10/500
4/4 - 0s - loss: 3.2827 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 11/500
4/4 - 0s - loss: 3.2440 - accuracy: 0.3333 - 17ms/epoch - 4ms/step
Epoch 12/500
4/4 - 0s - loss: 3.2225 - accuracy: 0.3333 - 18ms/epoch - 4ms/step
Epoch 13/500
4/4 - 0s - loss: 3.2043 - accuracy: 

Epoch 104/500
4/4 - 0s - loss: 0.5431 - accuracy: 0.9636 - 18ms/epoch - 5ms/step
Epoch 105/500
4/4 - 0s - loss: 0.5245 - accuracy: 0.9697 - 18ms/epoch - 4ms/step
Epoch 106/500
4/4 - 0s - loss: 0.5047 - accuracy: 0.9727 - 17ms/epoch - 4ms/step
Epoch 107/500
4/4 - 0s - loss: 0.4899 - accuracy: 0.9606 - 18ms/epoch - 5ms/step
Epoch 108/500
4/4 - 0s - loss: 0.4636 - accuracy: 0.9758 - 18ms/epoch - 5ms/step
Epoch 109/500
4/4 - 0s - loss: 0.4498 - accuracy: 0.9758 - 18ms/epoch - 5ms/step
Epoch 110/500
4/4 - 0s - loss: 0.4297 - accuracy: 0.9758 - 17ms/epoch - 4ms/step
Epoch 111/500
4/4 - 0s - loss: 0.4126 - accuracy: 0.9727 - 17ms/epoch - 4ms/step
Epoch 112/500
4/4 - 0s - loss: 0.3939 - accuracy: 0.9758 - 17ms/epoch - 4ms/step
Epoch 113/500
4/4 - 0s - loss: 0.3836 - accuracy: 0.9818 - 18ms/epoch - 4ms/step
Epoch 114/500
4/4 - 0s - loss: 0.3683 - accuracy: 0.9788 - 18ms/epoch - 4ms/step
Epoch 115/500
4/4 - 0s - loss: 0.3498 - accuracy: 0.9818 - 18ms/epoch - 4ms/step
Epoch 116/500
4/4 - 0s - los

4/4 - 0s - loss: 0.0041 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 206/500
4/4 - 0s - loss: 0.0035 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 207/500
4/4 - 0s - loss: 0.0031 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 208/500
4/4 - 0s - loss: 0.0030 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 209/500
4/4 - 0s - loss: 0.0029 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 210/500
4/4 - 0s - loss: 0.0030 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 211/500
4/4 - 0s - loss: 0.0026 - accuracy: 1.0000 - 18ms/epoch - 5ms/step
Epoch 212/500
4/4 - 0s - loss: 0.0024 - accuracy: 1.0000 - 18ms/epoch - 5ms/step
Epoch 213/500
4/4 - 0s - loss: 0.0023 - accuracy: 1.0000 - 19ms/epoch - 5ms/step
Epoch 214/500
4/4 - 0s - loss: 0.0022 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 215/500
4/4 - 0s - loss: 0.0022 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 216/500
4/4 - 0s - loss: 0.0019 - accuracy: 1.0000 - 19ms/epoch - 5ms/step
Epoch 217/500
4/4 - 0s - loss: 0.0019 - ac

Epoch 303/500
4/4 - 0s - loss: 1.4148e-05 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 304/500
4/4 - 0s - loss: 1.3616e-05 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 305/500
4/4 - 0s - loss: 1.3730e-05 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 306/500
4/4 - 0s - loss: 1.1961e-05 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 307/500
4/4 - 0s - loss: 1.1390e-05 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 308/500
4/4 - 0s - loss: 1.1068e-05 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 309/500
4/4 - 0s - loss: 1.0953e-05 - accuracy: 1.0000 - 18ms/epoch - 5ms/step
Epoch 310/500
4/4 - 0s - loss: 1.0189e-05 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 311/500
4/4 - 0s - loss: 9.9629e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 312/500
4/4 - 0s - loss: 9.4333e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 313/500
4/4 - 0s - loss: 9.5305e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 314/500
4/4 - 0s - loss: 8.4772e-06 - accuracy: 1.0000 - 16

Epoch 400/500
4/4 - 0s - loss: 1.2571e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 401/500
4/4 - 0s - loss: 1.2416e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 402/500
4/4 - 0s - loss: 1.2188e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 403/500
4/4 - 0s - loss: 1.2123e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 404/500
4/4 - 0s - loss: 1.1932e-06 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 405/500
4/4 - 0s - loss: 1.1834e-06 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 406/500
4/4 - 0s - loss: 1.1755e-06 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 407/500
4/4 - 0s - loss: 1.1628e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 408/500
4/4 - 0s - loss: 1.1520e-06 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 409/500
4/4 - 0s - loss: 1.1383e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 410/500
4/4 - 0s - loss: 1.1220e-06 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 411/500
4/4 - 0s - loss: 1.1083e-06 - accuracy: 1.0000 - 17

Epoch 497/500
4/4 - 0s - loss: 5.7365e-07 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 498/500
4/4 - 0s - loss: 5.7148e-07 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 499/500
4/4 - 0s - loss: 5.6931e-07 - accuracy: 1.0000 - 18ms/epoch - 4ms/step
Epoch 500/500
4/4 - 0s - loss: 5.6570e-07 - accuracy: 1.0000 - 18ms/epoch - 5ms/step
학습시간 :  {10.347182273864746}


In [99]:
model.save('data/seq2seq.h5')